<a href="https://colab.research.google.com/github/carmea2025-dev/conversor-colab/blob/main/C%C3%B3pia_de_Conversor_BPA_Com_Botoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Projeto BAHIA QUE PRODUZ E ALIMENTA - Plano de Ação { display-mode: "form" }

from IPython.display import display, HTML, clear_output
import pandas as pd
import io
import requests
from google.colab import files
import ipywidgets as widgets
from openpyxl import load_workbook
from openpyxl.styles import PatternFill, Alignment
from datetime import datetime, date
import unicodedata
import re

# ============================================================
# TÍTULO VISÍVEL MESMO COM CÓDIGO OCULTO
# ============================================================
display(HTML("""
<h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
</h2>
<hr style="border:1px solid #ccc;">
"""))

# ============================================================
# URLs DOS ARQUIVOS NO GITHUB
# ============================================================
url_mapeamento = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/Mapeamento_Colunas.xlsx"
url_modelo = "https://raw.githubusercontent.com/carmea2025-dev/conversor-colab/main/PLANO%20DE%20AÇÃO%20-%20BPA%20-%20FINAL.xlsx"

# ============================================================
# FUNÇÃO PARA BAIXAR ARQUIVOS DO GITHUB
# ============================================================
def baixar_arquivo(url, nome_arquivo):
    r = requests.get(url)
    if r.status_code == 200:
        with open(nome_arquivo, "wb") as f:
            f.write(r.content)
        print(f"✅ {nome_arquivo} baixado com sucesso.")
    else:
        print(f"❌ Erro ao baixar {nome_arquivo}. Verifique o link do GitHub.")

baixar_arquivo(url_mapeamento, "Mapeamento_Colunas.xlsx")
baixar_arquivo(url_modelo, "PLANO_DE_ACAO_BPA_FINAL.xlsx")

# ============================================================
# FUNÇÕES AUXILIARES PARA NORMALIZAÇÃO
# ============================================================
def normalize_text_full(s):
    """
    Normaliza texto: None -> '', remove acentos, deixa minúsculo,
    remove caracteres não alfanuméricos (mantém espaços), reduz múltiplos espaços.
    Retorna string sem acentos pronta para comparação.
    """
    if s is None:
        return ""
    s = str(s).strip().lower()
    # remove acentos
    s = ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')
    # mantém letras, números e espaços
    s = re.sub(r'[^a-z0-9\s]', '', s)
    # reduz múltiplos espaços e remove espaços nas extremidades
    s = re.sub(r'\s+', ' ', s).strip()
    return s

def parse_date_cell(value, wb):
    """Converte value (float/int datetime.date/datetime string) para datetime.date ou None"""
    if value is None or value == "":
        return None
    # já é datetime
    if isinstance(value, datetime):
        return value.date()
    if isinstance(value, date):
        return value
    # números (serial excel)
    if isinstance(value, (int, float)):
        try:
            from openpyxl.utils.datetime import from_excel
            dt = from_excel(value, wb.epoch)
            if isinstance(dt, datetime):
                return dt.date()
            elif isinstance(dt, date):
                return dt
        except Exception:
            # fallback via pandas (dias desde 1899-12-30)
            try:
                dt = pd.to_datetime(value, unit='d', origin='1899-12-30', errors='coerce')
                if pd.isna(dt):
                    return None
                return dt.date()
            except Exception:
                return None
    # strings: tenta parse com pandas ou formatos comuns
    if isinstance(value, str):
        value_strip = value.strip()
        for fmt in ("%d/%m/%Y", "%Y-%m-%d", "%d-%m-%Y", "%d/%m/%y"):
            try:
                return datetime.strptime(value_strip, fmt).date()
            except Exception:
                continue
        try:
            dt = pd.to_datetime(value_strip, dayfirst=True, errors='coerce')
            if pd.isna(dt):
                return None
            return dt.date()
        except Exception:
            return None
    return None

# ============================================================
# FUNÇÃO DE CONVERSÃO
# ============================================================
def converter_arquivo(uploaded_file):
    try:
        # Lê os arquivos de apoio
        mapeamento = pd.read_excel("Mapeamento_Colunas.xlsx", engine='openpyxl')
        modelo_df = pd.read_excel("PLANO_DE_ACAO_BPA_FINAL.xlsx", engine='openpyxl')

        nome_arquivo = list(uploaded_file.keys())[0]
        conteudo = uploaded_file[nome_arquivo]['content']

        # Detecta extensão do arquivo enviado
        if nome_arquivo.endswith(".xls"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='xlrd')
        elif nome_arquivo.endswith(".xlsx"):
            df_bpa = pd.read_excel(io.BytesIO(conteudo), engine='openpyxl')
        else:
            print("⚠️ Formato de arquivo não suportado. Envie .xls ou .xlsx.")
            return

        # Cria o dicionário de mapeamento
        col_map = dict(zip(mapeamento['Coluna_Origem'], mapeamento['Coluna_Destino']))

        # Aplica o mapeamento às colunas do BPA
        df_convertido = df_bpa.rename(columns=col_map)

        # Carrega o modelo com formatação
        wb = load_workbook("PLANO_DE_ACAO_BPA_FINAL.xlsx")
        ws = wb.active

        # Define as colunas fixas que determinam a estrutura e vínculo das ações
        colunas_fixas = ["Nº", "COMPONENTE", "SUBCOMPONENTE", "Responsabilidade", "Ação"]

        # Cria o dataframe final iniciando a partir do modelo (preserva estrutura)
        df_final = modelo_df.copy()

        # Garante que o DataFrame convertido tenha apenas colunas conhecidas
        colunas_modelo = list(modelo_df.columns)
        df_convertido = df_convertido[[c for c in df_convertido.columns if c in colunas_modelo]]

        # Mescla os dados do modelo (estrutura) com o BPA convertido (dados)
        if "Ação" in df_convertido.columns:
            df_final = pd.merge(
                modelo_df,
                df_convertido,
                on="Ação",
                how="left",
                suffixes=("", "_NOVO")
            )

            # Preenche as colunas do modelo com os dados novos
            for col in colunas_modelo:
                col_novo = col + "_NOVO"
                if col_novo in df_final.columns:
                    df_final[col] = df_final[col_novo].combine_first(df_final[col])
                    df_final.drop(columns=[col_novo], inplace=True, errors="ignore")

        # Atualiza os valores do df_final no ws
        for col_idx, col_name in enumerate(colunas_modelo, start=1):
            if col_name not in df_final.columns:
                continue
            valores = df_final[col_name].tolist()
            for row_idx, valor in enumerate(valores, start=2):
                ws.cell(row=row_idx, column=col_idx, value=valor)

        # ============================================================
        # APLICAÇÃO DA REGRA DE CORES NA COLUNA "Progresso"
        # ============================================================
        hoje = datetime.now().date()
        verde = PatternFill(start_color="92D050", end_color="92D050", fill_type="solid")
        amarelo = PatternFill(start_color="FFFF00", end_color="FFFF00", fill_type="solid")
        vermelho = PatternFill(start_color="FF0000", end_color="FF0000", fill_type="solid")
        centralizar = Alignment(horizontal="center", vertical="center")

        headers = [cell.value for cell in ws[1]]
        if "Progresso" not in headers:
            print("⚠️ Coluna 'Progresso' não encontrada no modelo.")
            # salvar e encerrar para evitar index error
            nome_saida = "PLANO DE AÇÃO - BPA - CONVERTIDO.xlsx"
            wb.save(nome_saida)
            files.download(nome_saida)
            return

        idx_progresso = headers.index("Progresso") + 1
        idx_data_inicio = headers.index("Data de Início Planejada") + 1 if "Data de Início Planejada" in headers else None
        idx_data_conclusao = headers.index("Data de Conclusão Planejada") + 1 if "Data de Conclusão Planejada" in headers else None

        # contadores (opcional, para relatório)
        cont_verde = cont_amarelo = cont_vermelho = 0

        for i in range(2, ws.max_row + 1):
            progresso_raw = ws.cell(i, idx_progresso).value
            data_inicio_raw = ws.cell(i, idx_data_inicio).value if idx_data_inicio else None
            data_conclusao_raw = ws.cell(i, idx_data_conclusao).value if idx_data_conclusao else None

            progresso = normalize_text_full(progresso_raw)
            data_inicio = parse_date_cell(data_inicio_raw, wb) if idx_data_inicio else None
            data_conclusao = parse_date_cell(data_conclusao_raw, wb) if idx_data_conclusao else None

            # garante que datas sejam datetime.date (parse_date_cell já faz isso)
            if isinstance(data_inicio, datetime):
                data_inicio = data_inicio.date()
            if isinstance(data_conclusao, datetime):
                data_conclusao = data_conclusao.date()

            celula = ws.cell(i, idx_progresso)
            celula.alignment = centralizar

            try:
                # NOVA REGRA EXPLÍCITA: se Progresso = "Concluída" -> verde
                if progresso in ["concluida", "concluida"]:
                    celula.fill = verde
                    cont_verde += 1
                    continue

                # Não iniciado -> usa Data de Início Planejada
                if progresso in ["nao iniciado", "naoiniciado", "nao_iniciado", "nao iniciado"]:
                    if not data_inicio:
                        # se não há data, não pinta (ou poderia pintar de cinza)
                        continue
                    if data_inicio > hoje:
                        celula.fill = verde
                        cont_verde += 1
                    elif data_inicio == hoje:
                        celula.fill = amarelo
                        cont_amarelo += 1
                    else:
                        celula.fill = vermelho
                        cont_vermelho += 1
                    continue

                # Em andamento -> usa Data de Conclusão Planejada
                if progresso in ["em andamento", "emandamento", "em_andamento", "em andamento"]:
                    if not data_conclusao:
                        continue
                    if data_conclusao > hoje:
                        celula.fill = verde
                        cont_verde += 1
                    elif data_conclusao == hoje:
                        celula.fill = amarelo
                        cont_amarelo += 1
                    else:
                        celula.fill = vermelho
                        cont_vermelho += 1
                    continue

                # Caso não reconheça o texto, não altera (opcional: pintar cinza)
            except Exception as e:
                print(f"⚠️ Linha {i}: erro ao aplicar regra ({e}) → Progresso: {progresso_raw}, Início: {data_inicio_raw}, Conclusão: {data_conclusao_raw}")

        # ============================================================
        # SALVAR E DOWNLOAD
        # ============================================================
        nome_saida = "PLANO DE AÇÃO - BPA - CONVERTIDO.xlsx"
        wb.save(nome_saida)
        print("✅ Conversão concluída com sucesso! Estrutura e formatação preservadas.")
        # resumo opcional
        print(f"Resumo: verdes={cont_verde}, amarelas={cont_amarelo}, vermelhas={cont_vermelho}")
        files.download(nome_saida)

    except Exception as e:
        print(f"❌ Erro durante a conversão: {e}")

# ============================================================
# INTERFACE NO COLAB
# ============================================================
def interface():
    upload_button = widgets.FileUpload(accept=".xls,.xlsx", multiple=False)
    convert_button = widgets.Button(description="Converter e Baixar", button_style="success")

    def ao_clicar(_):
        clear_output(wait=True)
        display(HTML("""
        <h2 style="text-align:center; color:#2c3e50; font-family:'Trebuchet MS', sans-serif;">
        📘 Projeto <b>BAHIA QUE PRODUZ E ALIMENTA</b> - Plano de Ação
        </h2>
        <hr style="border:1px solid #ccc;">
        """))
        display(upload_button, convert_button)
        if upload_button.value:
            converter_arquivo(upload_button.value)
        else:
            print("⚠️ Envie um arquivo BPA primeiro.")

    convert_button.on_click(ao_clicar)
    display(upload_button, convert_button)

# ============================================================
# EXECUTA A INTERFACE
# ============================================================
interface()


FileUpload(value={'PLANO DE AÇÃO - BPA.xlsx': {'metadata': {'name': 'PLANO DE AÇÃO - BPA.xlsx', 'type': 'appli…

Button(button_style='success', description='Converter e Baixar', style=ButtonStyle())

✅ Conversão concluída com sucesso! Estrutura e formatação preservadas.
Resumo: verdes=55, amarelas=3, vermelhas=62


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>